In [1]:
import numpy as np

In [2]:
input_data=np.load("CK+dataset.npy")
label=np.load("CK+label.npy")

In [3]:
input_data.shape

(327, 640, 480)

In [4]:
label.shape

(327,)

In [5]:
label=label-1

In [8]:
import matplotlib.pyplot as plt
import numpy as np
import random
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import argparse
import numpy as np
import time
from copy import deepcopy
import seaborn as sns 
import matplotlib.pyplot as plt

In [9]:
input_data=torch.FloatTensor(input_data)
label=torch.LongTensor(label)

In [10]:
input_data.shape

torch.Size([327, 640, 480])

CNN Architecture

In [11]:
remake=input_data.unsqueeze(0)
remake=remake.transpose(0,1)

In [12]:
remake.shape

torch.Size([327, 1, 640, 480])

In [13]:
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
dataset = TensorDataset(remake,label)

In [14]:
trainset,valset=torch.utils.data.random_split(dataset,[277,50])
trainset,testset=torch.utils.data.random_split(trainset,[227,50])

partition={'train':trainset,'val':valset,'test':testset}

In [15]:
class CNN(nn.Module):
    
    def __init__(self):
        super(CNN, self).__init__()
        
        self.conv1=nn.Conv2d(in_channels=1,
                            out_channels=32,
                            kernel_size=6,
                            stride=2,
                            padding=2) # 32 24 24
        self.conv2=nn.Conv2d(in_channels=32, 
                            out_channels=64,
                            kernel_size=5,
                            stride=2,
                            padding=2) # 64 60 64
        self.conv3=nn.Conv2d(in_channels=64,
                            out_channels=128,
                            kernel_size=3,
                            stride=1,
                            padding=1) # 25 30 128
        self.act=nn.ReLU()
        self.maxpool1=nn.MaxPool2d(kernel_size=2,
                                  stride=2)
        self.fc=nn.Linear(38400,7)
    def forward(self,x):
        x=self.conv1(x)
        x=self.act(x)
        x=self.maxpool1(x)
        x=self.conv2(x)
        x=self.act(x)
        x=self.maxpool1(x)
        x=self.conv3(x)
        x=self.act(x)
        x=self.maxpool1(x)
        x=x.view(x.size(0),-1)
        x=self.fc(x)
        return x

In [16]:
def dimension_check():
    net=CNN()
    x=torch.randn(1,1,640,480)
    y=net(x)
    print(y.size())

In [17]:
dimension_check()

torch.Size([1, 7])


In [18]:
def train(net,partition,optimizer,criterion):
    trainloader = torch.utils.data.DataLoader(partition['train'],
                                             batch_size=32,
                                             shuffle=True)
    net.train()
    correct = 0
    total =0
    train_loss = 0.0
    for i, data in enumerate(tqdm(trainloader, 0)):
        time.sleep(0.0000001)
        optimizer.zero_grad()
        inputs, labels = data
        inputs = inputs.cuda()
        labels = labels.cuda()
        outputs = net(inputs)
        #break
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        #if i % 200 == 0:
        #    print(correct,train_loss)
    train_loss = train_loss / len(trainloader)
    train_acc = 100 * correct / total
    return net, train_loss, train_acc

In [19]:
def train(net,partition,optimizer,criterion):
    trainloader = torch.utils.data.DataLoader(partition['train'],
                                             batch_size=32,
                                             shuffle=True)
    net.train()
    correct = 0
    total =0
    train_loss = 0.0
    for i, data in enumerate(tqdm(trainloader, 0)):
        time.sleep(0.0000001)
        optimizer.zero_grad()
        inputs, labels = data
        inputs = inputs.cuda()
        labels = labels.cuda()
        outputs = net(inputs)
        #break
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        #if i % 200 == 0:
        #    print(correct,train_loss)
    train_loss = train_loss / len(trainloader)
    train_acc = 100 * correct / total
    return net, train_loss, train_acc

In [20]:
def validate(net, partition, criterion):
    valloader = torch.utils.data.DataLoader(partition['val'], 
                                            batch_size=32, 
                                            shuffle=False, num_workers=2)
    net.eval()

    correct = 0
    total = 0
    val_loss = 0 
    with torch.no_grad():
        for data in tqdm(valloader):
            time.sleep(0.0000001)
            images, labels = data
            images = images.cuda()
            labels = labels.cuda()
            outputs = net(images)

            loss = criterion(outputs, labels)
            
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        val_loss = val_loss / len(valloader)
        val_acc = 100 * correct / total
    return val_loss, val_acc

In [21]:
from tqdm import tqdm, tqdm_notebook
from tqdm import trange
import time

In [23]:
seed = 123
np.random.seed(seed)
torch.manual_seed(seed)
net=CNN()
net.cuda()
criterion=nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.0015, weight_decay=0.00001)
train_losses=[]
val_losses=[]
train_accs=[]
val_accs=[]
for epoch in range(50):
    ts=time.time()
    net, train_loss,train_acc=train(net, partition, optimizer, criterion)
    val_loss, val_acc = validate(net, partition, criterion)
    te=time.time()
    print('time : {:2.2f}, train_loss : {:2.2f}, train_acc : {:2.2f},val_acc : {:2.2f},val_loss : {:2.2f}'.format(te-ts,train_loss,train_acc,val_acc,val_loss))
    train_loss=round(train_loss,2)
    val_loss=round(val_loss,2)
    train_acc=round(train_acc,2)
    val_acc=round(val_acc,2)
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_accs.append(train_acc)
    val_accs.append(val_acc)
test_acc = test(net,partition)
test_acc=round(test_acc,2)

 50%|█████     | 4/8 [00:00<00:00, 35.18it/s]

time : 0.33, train_loss : 2.01, train_acc : 19.82,val_acc : 34.00,val_loss : 1.85


 50%|█████     | 4/8 [00:00<00:00, 35.27it/s]

time : 0.31, train_loss : 1.84, train_acc : 22.47,val_acc : 16.00,val_loss : 1.82


 50%|█████     | 4/8 [00:00<00:00, 35.38it/s]

...

 50%|█████     | 4/8 [00:00<00:00, 34.59it/s]

time : 0.32, train_loss : 0.05, train_acc : 99.12,val_acc : 40.00,val_loss : 3.85


  0%|          | 0/2 [00:00<?, ?it/s]

time : 0.32, train_loss : 0.02, train_acc : 100.00,val_acc : 44.00,val_loss : 3.83


100%|██████████| 2/2 [00:00<00:00, 20.02it/s]


In [ ]:
np.save("/home/lab/양창희/Emotion Project/test_list_file/Basic_train_loss.npy",train_loss)
np.save("/home/lab/양창희/Emotion Project/test_list_file/Basic_val_loss.npy",val_loss)
np.save("/home/lab/양창희/Emotion Project/test_list_file/Basic_train_acc.npy",train_acc)
np.save("/home/lab/양창희/Emotion Project/test_list_file/Basic_val_acc.npy",val_acc)